In [ ]:
import numpy as np
import limp

In [ ]:
# Define the size of our problem:
N_students = 400 # tested up to 400 students
N_slots = 7 # how many total courses can a student take?
core_subjects = ['English', 'History', 'Math', 'Science', 'Latin']
N_levels = 3 # beginner/intermediate/advanced, or freshman/sophomore/junior/senior
N_electives = 20

# Derive some useful constants:
N_core_subjects = len(core_subjects)
N_elective_slots = N_slots - N_core_subjects
# This is unique "course numbers" -- multiple instances of one class may be offered
N_classes = N_core_subjects * N_levels + N_electives
core_starts = np.arange(N_core_subjects) * N_levels
core_ends = core_starts + N_levels
first_elective = core_ends.max()

# Randomly generate a set of classes for each student.
# Most core classes will be at the same level, and electives are at random.
rng1 = np.random.default_rng(12345)
# Students are mostly at a consistent level across their core classes, but occassionally differ
student_levels = np.where(
    rng1.uniform(size=(N_students, N_core_subjects)) < 0.2,
    rng1.choice(N_levels, size=(N_students, N_core_subjects)),
    rng1.choice(N_levels, size=(N_students, 1)),
)
student_core_classes = student_levels + core_starts[None,:]
student_electives = rng1.integers(N_electives, size=(N_students,N_elective_slots)) + first_elective
student_classes = np.concatenate([student_core_classes, student_electives], axis=1)
assert student_classes.shape == (N_students, N_slots)
# Shuffle each row independently, so some students take English first, others Math, others an elective
student_classes = rng1.permuted(student_classes, axis=1)

# One hot encode the schedule from student_classes: 
schedule = np.zeros((N_students, N_classes, N_slots), dtype=int)
schedule[np.arange(N_students)[:,None], student_classes, np.arange(N_slots)[None,:]] = 1
assert np.all(schedule.sum(axis=(1,2)) == N_slots), "Every student should take exactly N_slots classes"
assert np.all(schedule.sum(axis=1) == 1), "Exactly one class per student per time slot"

demand_at_time = schedule.sum(axis=0) # (N_classes, N_slots)
demand = demand_at_time.sum(axis=1) # how many students want each class

assignments = schedule.sum(axis=2) # (N_students, N_classes) without regard to time slot
co_demand = assignments.T @ assignments # how many students want to take that pair of classes
np.fill_diagonal(co_demand, 0) # diagonal was equal to demand, but we don't want to penalize it

## Exact recovery of full student schedule

In [ ]:
# Can we recover a fully satisfactory schedule for all students if we have no limits on class size?

p = limp.Problem()
D = p.binvar_array('d', schedule.shape) # is student s taking class c at time t?
print(D.size, "decision variables")

# Each student takes the same number of classes
for s in range(N_students):
    p.exactly(N_slots, D[s,:,:].flatten())

# Each student should have exactly 1 class scheduled per time slot
for s in range(N_students):
    for t in range(N_slots):
        p.exactly(1, D[s,:,t])

# Each student should get the classes they asked for (at some time),
# and no classes that they didn't ask for.
for s in range(N_students):
    for c in range(N_classes):
        p.exactly(assignments[s,c], D[s,c,:])

%time ans = p.solve(time_limit=10)
print(ans['soln'].message)
D_solve = np.vectorize(lambda x: ans['by_var'][x])(D).round().astype(int)

In [ ]:
# What if we limit class sizes to exactly what they were in the original schedule?
# Yes, even with no slack.  Much slower, even if we allow plenty of room for class sizes to grow!
slack = 2.00

for c in range(N_classes):
    for t in range(N_slots):
        p.at_most(slack*demand_at_time[c,t], D[:,c,t])

%time ans = p.solve(time_limit=10)
print(ans['soln'].message)
D_solve = np.vectorize(lambda x: ans['by_var'][x])(D).round().astype(int)

# Our found solution mostly matches the original schedule, though not exactly.
(D_solve == schedule).mean()

## Exact core schedule, best effort electives

In [ ]:
# Can we recover a fully satisfactory schedule for all students if we have no limits on class size?

p = limp.Problem()
D = p.binvar_array('d', schedule.shape) # is student s taking class c at time t?

# Each student takes the same number of classes
for s in range(N_students):
    p.exactly(N_slots, D[s,:,:].flatten())

# Each student should have exactly 1 class scheduled per time slot
for s in range(N_students):
    for t in range(N_slots):
        p.exactly(1, D[s,:,t])

# Each student should get the core classes they asked for (at some time).
# We will maximize the number of elective requests satisfied.
num_elec_satisfied = limp.Expr()
for s in range(N_students):
    for c in range(first_elective):
        p.exactly(assignments[s,c], D[s,c,:])
    for c in range(first_elective, N_classes):
        if assignments[s,c]:
            num_elec_satisfied += p.sum(D[s,c,:])

%time ans = p.maximize(num_elec_satisfied, time_limit=10)
print(ans['soln'].message)
print(f'Objective:      {ans['soln'].fun}')
print(f'Lower bound:    {ans['soln'].mip_dual_bound}    (gap = {ans['soln'].mip_gap})')
print(f'# variables:    {len(ans['vs'])}')
print(f'# constraints:  {len(ans['constraints'].A)}')
D_solve = np.vectorize(lambda x: ans['by_var'][x])(D).round().astype(int)

In [ ]:
# What if we limit class sizes to exactly what they were in the original schedule?
# With no slack, possible but slow.  Not as slow as exact satisfaction though.
# With 33% slack, possible and faster.
# With 50% slack, almost as fast as no constraint on class sizes.
slack = 1.33

for c in range(N_classes):
    for t in range(N_slots):
        p.at_most(slack*demand_at_time[c,t], D[:,c,t])

%time ans = p.maximize(num_elec_satisfied, time_limit=10)
print(ans['soln'].message)
print(f'Objective:      {ans['soln'].fun}')
print(f'Lower bound:    {ans['soln'].mip_dual_bound}    (gap = {ans['soln'].mip_gap})')
print(f'# variables:    {len(ans['vs'])}')
print(f'# constraints:  {len(ans['constraints'].A)}')
D_solve = np.vectorize(lambda x: ans['by_var'][x])(D).round().astype(int)

# Course offering times limited by teacher availability

This is exactly analogous to the "conference scheduling" problem.
We want to assign classes to times such that we have sufficient teachers available at that time.

In [ ]:
class_size = 20

N_core_sections = np.ceil((demand[:first_elective]) / class_size).astype(int)

# Elective teachers will teach the classes with the highest demand,
# potentially leaving some demand unsatisfied.
# This greedy algo effectively minimizes the square of unsatisfied demand.
N_elective_teachers_total = np.ceil((N_elective_slots * N_students) / (class_size * N_slots)) # assume these teachers can cover ANY elective
tot = N_elective_teachers_total * N_slots
dem = demand[first_elective:].copy()
sup = np.zeros_like(dem)
while sup.sum() < tot:
    i = np.argmax(dem)
    sup[i] += 1
    dem[i] -= class_size

N_elective_sections = sup
N_sections = np.concatenate([N_core_sections, N_elective_sections])

# N_slots-1 because we need to plan for each teacher to have a planning period!
# (Though right now we don't have a constraint to enforce it.)
N_core_teachers = [np.ceil(N_sections[core_starts[s]:core_ends[s]].sum() / (N_slots-1)) for s in range(N_core_subjects)]

capacity = N_sections * class_size
assert np.all((capacity >= demand)[:first_elective])

In [ ]:
p = limp.Problem()
# How many sections of each class will be taught at each time?
K = p.intvar_array('k', shape=(N_classes, N_slots), lower=0, upper=N_sections[:,None])

# Across all time slots, at least the target number of sections must be scheduled for each class
# We *may* schedule extra if we have staff available, so all students have a class to be in
for c in range(N_classes):
    p.constraint(N_sections[c], p.sum(K[c,:]))

# For each time and core subject, the number of sections cannot exceed the number of teachers for that subject
for s in range(N_core_subjects):
    for t in range(N_slots):
        p.constraint(p.sum(K[core_starts[s]:core_ends[s], t]), N_core_teachers[s])

# For each time, the number of elective sections cannot exceed the number of electives teachers
for t in range(N_slots):
    p.constraint(p.sum(K[first_elective:, t]), N_elective_teachers_total)

# At each time, we must have enough sections scheduled so all students can be in a class
for t in range(N_slots):
    p.constraint(N_students, p.sum(K[:,t]) * class_size)

# Reduce the cost of conflict when there are multiple sections of one or both classes,
# both because *some* conflict is more likely and some work-around is more likely possible.
nsec1 = np.maximum(1, N_sections) # avoid divide-by-zero
norm_co_demand = co_demand / (nsec1[:,None] @ nsec1[None,:])
assert np.all(norm_co_demand == norm_co_demand.T), "Co-demand should be symmetric"

# Minimize the co-demand of classes scheduled during the same time period
conflicts = limp.Expr()
for c1 in range(N_classes):
    for c2 in range(c1+1, N_classes):
        for t in range(N_slots):
            # I really want the product of these two numbers,
            # but I don't think I can compute that for MILP.
            # However min() acts like a product as long as one of them is <= 1,
            # which is very likely to be the case while this problem is small.
            # For a true product effect, I probably have to switch to binary vars for each section of each class.
            conflicts += norm_co_demand[c1,c2] * p.min([K[c1,t], K[c2,t]])

# After 5 min, we know true minimum is in [37, 194] for 100 students, class size 20
%time ans = p.minimize(conflicts, time_limit=30)
print(ans['soln'].message)
print(f'Objective:      {ans['soln'].fun}')
print(f'Lower bound:    {ans['soln'].mip_dual_bound}    (gap = {ans['soln'].mip_gap})')
print(f'# variables:    {len(ans['vs'])}')
print(f'# constraints:  {len(ans['constraints'].A)}')

K_solve = np.vectorize(lambda x: ans['by_var'][x])(K).round().astype(int)
supply_at_time = K_solve * class_size

## Exact core schedule, best effort electives, constrained by teacher schedule

In [ ]:
# Can we recover a fully satisfactory schedule for all students if we have no limits on class size?

p = limp.Problem()
D = p.binvar_array('d', schedule.shape) # is student s taking class c at time t?
# This is a really inefficient formulation, because many classes aren't offered at every time.
# But some of our constraints make it trivial to deduce those variables must be zero,
# so the proliferation of unnecessary variables doesn't seem to hurt us much.
# The alternative would be to have some sort of sparse array of variables,
# or replace some with a constant 0 and modify constraint functions to allow a mix of Vars and constants.

# Each student takes the same number of classes
# (This constraint isn't actually necessary, it's implied by exactly one class per time slot.)
for s in range(N_students):
    p.exactly(N_slots, D[s,:,:].flatten())

# Each student should have exactly 1 class scheduled per time slot
for s in range(N_students):
    for t in range(N_slots):
        p.exactly(1, D[s,:,t])

# Limit on the number of students in each class (depending how many sections at this time)
for c in range(N_classes):
    for t in range(N_slots):
        p.at_most(supply_at_time[c,t], D[:,c,t])

num_elec_satisfied = limp.Expr()
for s in range(N_students):
    # Each student should get the core classes they asked for (at some time).
    for c in range(first_elective):
        p.exactly(assignments[s,c], D[s,c,:])
    # We will maximize the number of elective requests satisfied.
    for c in range(first_elective, N_classes):
        if assignments[s,c]:
            num_elec_satisfied += p.sum(D[s,c,:])
    # # Best effort on all classes, not just electives:
    # for c in range(N_classes):
    #     if assignments[s,c]:
    #         num_elec_satisfied += p.sum(D[s,c,:])

num_elec_unsatisfied = (N_students * N_elective_slots) - num_elec_satisfied

# Also try to minimize wasted seats / extra sessions.
# So far, this seems not to make a difference to the answer.
wasted_seats = limp.Expr()
for c in range(N_classes):
    for t in range(N_slots):
        # Due to constraints above, we know this will always be >= 0
        wasted_seats += (supply_at_time[c,t] - p.sum(D[:,c,t]))

%time ans = p.minimize(num_elec_unsatisfied + 0.2*wasted_seats, time_limit=15)
print(ans['soln'].message)
print(f'Objective:      {num_elec_unsatisfied.eval(ans['by_var'])}')
# These values are correct to within a constant, because objective is just a dot product!
print(f'Raw objective:  {ans['soln'].fun}')
print(f'Lower bound:    {ans['soln'].mip_dual_bound}    (gap = {ans['soln'].mip_gap})')
print(f'# variables:    {len(ans['vs'])}')
print(f'# constraints:  {len(ans['constraints'].A)}')
D_solve = np.vectorize(lambda x: ans['by_var'][x])(D).round().astype(int)

In [ ]:
# Unused seats:
used_at_time = D_solve.sum(axis=0)
unused_at_time = supply_at_time - used_at_time
frac_unused = (unused_at_time.sum(axis=1) / demand).round(3)
np.sort(frac_unused)
unused_at_time.sum(axis=1)

# TODO
- [ ] float -> Real
- [ ] add type aliases Vars, VarLike, ExprOrVars, etc
- [ ] implement _extract_consts(vs)
- [ ] allow at_least(), at_most(), exactly() to take ExprOrVars
- [ ] during minimize(), create eqvar('_intercept', ...) to maintain objective's user-defined value